In [1]:
from datetime import date
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import yfinance as yf
df = pd.read_csv("../Resources/btcjoin.csv", parse_dates=['date'])
btc_df = yf.download('BTC-USD')
btc_df = btc_df.reset_index()
btc_df = btc_df.loc[(btc_df['Date'] > '2022-10-25')]
btc_df['Close']=btc_df['Close'].astype("float")
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype("float")
btc_df = btc_df.rename(columns={"Close": "price", "Date":"date"})
df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = df.rename(columns={"value": "wallets"})
df = df.drop(columns=['volume','change', 'low', 'high', 'open','Open','High','Low','Adj Close', 'Volume', 'Unnamed: 0', "wallets", "address", "mined"])

[*********************100%***********************]  1 of 1 completed


In [2]:
df['200D'] = df['price'].rolling(200).mean()
df['300D'] = df['price'].rolling(300).mean()
df['50D'] = df['price'].rolling(50).mean()
df = df.dropna()
df

,date,price,200D,300D,50D
299,2011-07-21,13.600000,5.665500,3.844333,16.604000
300,2011-07-22,13.700000,5.732500,3.889667,16.666000
301,2011-07-23,13.700000,5.799500,3.935000,16.654000
302,2011-07-24,14.000000,5.868000,3.981333,16.556000
303,2011-07-25,14.100000,5.937000,4.028000,16.504000
...,...,...,...,...,...
4422,2022-12-01,16967.132812,21579.920225,27838.111483,18397.432898
4423,2022-12-02,17088.660156,21516.118525,27757.033350,18351.610102
4424,2022-12-03,16908.236328,21448.472207,27672.086805,18306.138828
4425,2022-12-04,17130.486328,21390.706139,27583.075426,18267.374555


In [3]:
df['meanavge'] = (df['200D'] + df['300D'] + df['50D'] )/3

In [4]:
df = df.drop(columns=['200D','300D', '50D'])
df['meanvalue'] = df["price"] - df["meanavge"]
df['status'] = df['meanvalue'].apply(lambda x: '1' if x > 0 else '0')
df['status']=df['status'].astype("object")

In [5]:
df['price-meanavge']=df['price'] - df['meanavge']
df['move%'] = df['price-meanavge']/(df['price'] + df['meanavge'])

In [6]:
bins = [-0.43, -0.1, 0, 0.1, 0.43]
group_names = ["Severely Oversold","Oversold", "Neutral","Overbought"]
df["Valuation"] = pd.cut(df["move%"], bins, labels=group_names)
df

,date,price,meanavge,meanvalue,status,price-meanavge,move%,Valuation
299,2011-07-21,13.600000,8.704611,4.895389,1,4.895389,0.219479,Overbought
300,2011-07-22,13.700000,8.762722,4.937278,1,4.937278,0.219799,Overbought
301,2011-07-23,13.700000,8.796167,4.903833,1,4.903833,0.217985,Overbought
302,2011-07-24,14.000000,8.801778,5.198222,1,5.198222,0.227974,Overbought
303,2011-07-25,14.100000,8.823000,5.277000,1,5.277000,0.230205,Overbought
...,...,...,...,...,...,...,...,...
4422,2022-12-01,16967.132812,22605.154869,-5638.022056,0,-5638.022056,-0.142474,Severely Oversold
4423,2022-12-02,17088.660156,22541.587326,-5452.927169,0,-5452.927169,-0.137595,Severely Oversold
4424,2022-12-03,16908.236328,22475.565947,-5567.329618,0,-5567.329618,-0.141361,Severely Oversold
4425,2022-12-04,17130.486328,22413.718706,-5283.232378,0,-5283.232378,-0.133603,Severely Oversold


In [7]:
pricefrommean = df.meanvalue.iloc[-1].round(2)
pricefrommean

-5371.36

In [8]:
currentzone = df.Valuation.iloc[-1]
currentzone

'Severely Oversold'

In [12]:
# Average length of days price stayed under meanaverage or undervalued/severely oversold
delta1cycle = df.index[df['date']=='2015-09-15'].tolist()[0] - df.index[df['date']=='2013-12-04'].tolist()[0]
delta2cycle = df.index[df['date']=='2019-04-2'].tolist()[0] - df.index[df['date']=='2017-12-17'].tolist()[0] 
averageunder = int((delta1cycle+delta2cycle)/2)
averageunder


560

In [13]:
# Average length of days from previous cycle peak to reach new all time high
delta1fromp2p = df.index[df['date']=='2017-03-17'].tolist()[0] - df.index[df['date']=='2013-12-04'].tolist()[0]
delta2fromp2p = df.index[df['date']=='2020-11-29'].tolist()[0] - df.index[df['date']=='2017-12-17'].tolist()[0] 
averagep2p = int((delta1fromp2p+delta2fromp2p)/2)
averagep2p


1138

In [ ]:
# Days since last all time high
sincealltimehigh = df.index[-1] - df.index[df['price']==df.price.max()].tolist()[0]
sincealltimehigh